<a href="https://colab.research.google.com/github/cksdlakstp12/deep_learning_study/blob/main/PyTorch%EB%A1%9C_YOLO%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import cv2
import os
import os.path as osp
import pickle as pkl
import pandas as pd
import random

from urllib import request

cfg 파일 다운로드

In [ ]:
!git clone https://github.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch.git

Cloning into 'YOLO_v3_tutorial_from_scratch'...
remote: Enumerating objects: 95, done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 95
Unpacking objects: 100% (95/95), done.


In [ ]:
cfg_url = "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
dog_cycle_car_img_url = "https://raw.githubusercontent.com/ayooshkathuria/pytorch-yolo-v3/master/dog-cycle-car.png"
yolo_weights_url = "https://pjreddie.com/media/files/yolov3.weights"

saved_cfg_name = "yolov3.cfg"
saved_img_name = "dog-cycle-car.png"
saved_weights_name = "yolov3.weights"

request.urlretrieve(cfg_url,saved_cfg_name)
request.urlretrieve(dog_cycle_car_img_url,saved_img_name)
request.urlretrieve(yolo_weights_url,saved_weights_name)

('yolov3.weights', <http.client.HTTPMessage at 0x7fc96d535b90>)

# darknet 계층 구현

In [ ]:
def parse_cfg(cfgfile):
  file = open(cfgfile, "r")
  lines = file.read().split("\n")
  lines = [x for x in lines if len(x) > 0]
  lines = [x for x in lines if x[0] != "#"]
  lines = [x.rstrip().lstrip() for x in lines]

  block = {}
  blocks = []

  for line in lines:
    if line[0] == "[":
      if len(block) != 0:
        blocks.append(block)
        block = {}
      block["type"] = line[1:-1].rstrip()
    else:
      key, value = line.split("=")
      block[key.rstrip()] = value.lstrip()
  blocks.append(block)

  return blocks

In [ ]:
blocks = parse_cfg("/content/yolov3.cfg")
print(blocks)

[{'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '608', 'height': '608', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '32', 'size': '1', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '1', 'pad': '1', 'activation': 'leaky'}, {'type': 'shortcut', 'from': '-3', 'activation': 'linear'}, {'type': 'convolutional', 'batch_normalize': '1', 'filters': '128', 'siz

In [ ]:
class EmptyLayer(nn.Module):
  def __init__(self):
    super().__init__()

class DetectionLayer(nn.Module):
  def __init__(self, anchors):
    super().__init__()
    self.anchors = anchors

In [ ]:
def create_modules(blocks):
  net_info = blocks[0]
  module_list = nn.ModuleList()
  prev_filters = 3
  output_filters = []

  for index, x in enumerate(blocks[1:]):
    module = nn.Sequential()

    if (x["type"] == "convolutional"):
      try:
        batch_normalize = int(x["batch_normalize"])
        bias = False
      except:
        batch_normalize = 0
        bias = True

      filters = int(x["filters"])
      padding = int(x["pad"])
      kernel_size = int(x["size"])
      stride = int(x["stride"])

      if padding: pad = (kernel_size - 1) // 2
      else: pad = 0

      conv = nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias=bias)
      module.add_module(f"conv_{index}", conv)

      if batch_normalize:
        bn = nn.BatchNorm2d(filters)
        module.add_module(f"batch_norm_{index}", bn)

      if x["activation"] == "leaky":
        activn = nn.LeakyReLU(0.1, inplace=True)
        module.add_module(f"leaky_{index}", activn)

    elif (x["type"] == "upsample"):
      stride = int(x["stride"])
      upsample = nn.Upsample(scale_factor=stride, mode="bilinear")
      module.add_module(f"upsample_{index}", upsample)

    elif (x["type"] == "route"):
      x["layers"] = x["layers"].split(",")

      start = int(x["layers"][0])

      try: end = int(x["layers"][1])
      except: end = 0

      if start > 0: start = start - index
      if end > 0: end = end - index

      route = EmptyLayer()
      module.add_module(f"route_{index}", route)

      if end < 0: filters = output_filters[index + start] + output_filters[index + end]
      else: filters = output_filters[index + start]

    elif x["type"] == "shortcut":
      shortcut = EmptyLayer()
      module.add_module(f"shortcut_{index}", shortcut)

    elif x["type"] == "yolo":
      mask = x["mask"].split(",")
      mask = [int(x) for x in mask]

      anchors = x["anchors"].split(",")
      anchors = [int(a) for a in anchors]
      anchors = [(anchors[i], anchors[i+1]) for i in range(0, len(anchors), 2)]
      anchors = [anchors[i] for i in mask]

      detection = DetectionLayer(anchors)
      module.add_module(f"Detection_{index}", detection)

    module_list.append(module)
    prev_filters = filters
    output_filters.append(filters)

  return (net_info, module_list)

In [ ]:
blocks = parse_cfg("/content/yolov3.cfg")
model = create_modules(blocks)
print(model)

({'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '608', 'height': '608', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (2): Sequential(
    (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  

# YOLO 신경망 구조 구현

In [ ]:
class Darknet(nn.Module):
  def __init__(self, cfgfile):
    super(Darknet, self).__init__()
    self.blocks = parse_cfg(cfgfile)
    self.net_info, self.module_list = create_modules(self.blocks)

  def forward(self, x, CUDA):
    modules = self.blocks[1:]
    outputs = {}

    write = 0
    for i, module in enumerate(modules):
      module_type = (module["type"])
      
      if module_type == "convolutional" or module_type == "upsample":
        x = self.module_list[i](x)
      
      elif module_type == "route":
        layers = module["layers"]
        layers = [int(a) for a in layers]

        if (layers[0]) > 0:
          layers[0] = layers[0] - i

        if len(layers) == 1:
          x = outputs[i + (layers[0])]

        else:
          if (layers[1]) > 0:
            layers[1] = layers[1] - i

          map1 = outputs[i + layers[0]]
          map2 = outputs[i + layers[1]]

          x = torch.cat((map1, map2), 1)
      
      elif module_type == "shortcut":
        from_ = int(module["from"])
        x = outputs[i-1] + outputs[i+from_]

      elif module_type == "yolo":
        anchors = self.module_list[i][0].anchors
        inp_dim = int(self.net_info["height"])
        num_classes = int(module["classes"])
        
        x = x.data
        x = predict_transform(x, inp_dim, anchors, num_classes, CUDA)
        if not write:
          detections = x
          write = 1
        else:
          detections = torch.cat((detections, x), 1)
        
      outputs[i] = x
    
    return detections

  def load_weights(self, weightfile):
    with open(weightfile, "rb") as fp:
      header = np.fromfile(fp, dtype = np.int32, count=5)
      self.header = torch.from_numpy(header)
      self.seen = self.header[3]

      weights = np.fromfile(fp, dtype=np.float32)

      ptr = 0
      for i in range(len(self.module_list)):
        module_type = self.blocks[i+1]["type"]

        if module_type == "convolutional":
          model = self.module_list[i]
          try:
            batch_normalize = int(self.blocks[i+1]["batch_normalize"])
          except:
            batch_normalize = 0

          conv = model[0]

          if batch_normalize:
            bn = model[1]

            num_bn_biases = bn.bias.numel()

            bn_biases = torch.from_numpy(weights[ptr:ptr+num_bn_biases])
            ptr += num_bn_biases

            bn_weights = torch.from_numpy(weights[ptr:ptr+num_bn_biases])
            ptr += num_bn_biases

            bn_running_mean = torch.from_numpy(weights[ptr:ptr+num_bn_biases])
            ptr += num_bn_biases

            bn_running_var = torch.from_numpy(weights[ptr:ptr+num_bn_biases])
            ptr += num_bn_biases

            bn_biases = bn_biases.view_as(bn.bias.data)
            bn_weights = bn_weights.view_as(bn.weight.data)
            bn_running_mean = bn_running_mean.view_as(bn.running_mean)
            bn_running_var = bn_running_var.view_as(bn.running_var)

            bn.bias.data.copy_(bn_biases)
            bn.weight.data.copy_(bn_weights)
            bn.running_mean.copy_(bn_running_mean)
            bn.running_var.copy_(bn_running_var)
          
          else:
            num_biases = conv.bias.numel()

            conv_biases = torch.from_numpy(weights[ptr:ptr+num_biases])
            ptr = ptr + num_biases

            conv_biases = conv_biases.view_as(conv.bias.data)
            conv.bias.data.copy_(conv_biases)

          num_weights = conv.weight.numel()
          
          conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
          ptr = ptr + num_weights

          conv_weights = conv_weights.view_as(conv.weight.data)
          conv.weight.data.copy_(conv_weights)

In [ ]:
def predict_transform(prediction, inp_dim, anchors, num_classes, CUDA=True):
  batch_size = prediction.size(0)
  stride = inp_dim / prediction.size(2)
  grid_size = int(inp_dim // stride)
  stride = int(stride)
  bbox_attrs = 5 + num_classes
  num_anchors = len(anchors)
  
  prediction = prediction.view(batch_size, bbox_attrs * num_anchors, grid_size * grid_size)
  prediction = prediction.transpose(1,2).contiguous()
  prediction = prediction.view(batch_size, grid_size * grid_size * num_anchors, bbox_attrs)

  anchors = [(a[0]/stride, a[1]/stride) for a in anchors]

  prediction[:,:,0] = torch.sigmoid(prediction[:,:,0])
  prediction[:,:,1] = torch.sigmoid(prediction[:,:,1])
  prediction[:,:,4] = torch.sigmoid(prediction[:,:,4])

  grid = np.arange(grid_size)
  a, b = np.meshgrid(grid, grid)
  x_offset = torch.FloatTensor(a).view(-1, 1)
  y_offset = torch.FloatTensor(b).view(-1, 1)

  if CUDA:
    x_offset = x_offset.cuda()
    y_offset = y_offset.cuda()

  x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1, num_anchors).view(-1, 2).unsqueeze(0)
  prediction[:,:,:2] += x_y_offset

  anchors = torch.FloatTensor(anchors)

  if CUDA:
    anchors = anchors.cuda()
  
  anchors = anchors.repeat(grid_size * grid_size, 1).unsqueeze(0)
  prediction[:,:,2:4] = torch.exp(prediction[:,:,2:4]) * anchors
  prediction[:,:,5:5 + num_classes] = torch.sigmoid((prediction[:,:,5:5+num_classes]))
  prediction[:,:,:4] *= stride

  return prediction

In [ ]:
# 순전파 테스트

def get_test_input():
  img = cv2.imread("dog-cycle-car.png")
  img = cv2.resize(img, (416, 416))
  img_ = img[:,:,::-1].transpose((2,0,1))
  img_ = img_[np.newaxis,:,:,:]/255.0
  img_ = torch.from_numpy(img_).float()
  img_ = Variable(img_)
  return img_

model = Darknet("yolov3.cfg")
inp = get_test_input()
pred = model(inp, torch.cuda.is_available())
print(pred)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


tensor([[[2.4213e+01, 2.6042e+01, 1.0305e+02,  ..., 4.8836e-01,
          4.7237e-01, 4.9009e-01],
         [2.6077e+01, 2.4159e+01, 1.2255e+02,  ..., 4.5722e-01,
          5.5009e-01, 5.5051e-01],
         [2.0923e+01, 2.8121e+01, 3.6447e+02,  ..., 4.3059e-01,
          3.5980e-01, 4.7601e-01],
         ...,
         [5.6651e+02, 5.6618e+02, 4.9607e+00,  ..., 5.2353e-01,
          4.4016e-01, 5.6673e-01],
         [5.6639e+02, 5.6696e+02, 2.1043e+01,  ..., 5.4348e-01,
          5.0600e-01, 4.8549e-01],
         [5.6597e+02, 5.6600e+02, 3.8086e+01,  ..., 5.3822e-01,
          5.2525e-01, 4.3345e-01]]])


In [ ]:
# load_weights 테스트

model = Darknet("yolov3.cfg")
model.load_weights("yolov3.weights")

In [ ]:
def write_results(prediction, confidence, num_classes, nms_conf = 0.4):
  conf_mask = (prediction[:,:,4] > confidence).float().unsqueeze(2)
  prediction = prediction * conf_mask

  box_corner = prediction.new(prediction.shape)
  box_corner[:,:,0] = (prediction[:,:,0] - prediction[:,:,2]/2)
  box_corner[:,:,1] = (prediction[:,:,1] - prediction[:,:,3]/2)
  box_corner[:,:,2] = (prediction[:,:,0] + prediction[:,:,2]/2)
  box_corner[:,:,3] = (prediction[:,:,1] + prediction[:,:,3]/2)
  prediction[:,:,:4] = box_corner[:,:,:4]

  batch_size = prediction.size(0)
  write = 0
  for ind in range(batch_size):
    image_pred = prediction[ind]

    # max_conf, max_conf_index = torch.max(image_pred[:,5:5+num_classes], 1)
    max_conf, max_conf_index = map(lambda x: x.float().unsqueeze(1), torch.max(image_pred[:,5:5+num_classes], 1))
    # max_conf = max_conf.float().unsqueeze(1)
    # max_conf_index = max_conf_index.float().unsqueeze(1)
    seq = (image_pred[:,:5], max_conf, max_conf_index)
    image_pred = torch.cat(seq, 1)

    non_zero_ind = (torch.nonzero(image_pred[:, 4]))
    try: image_pred_ = image_pred[non_zero_ind.squeeze(), :].view(-1, 7)
    except: continue

    if image_pred_.shape[0] == 0: continue

    img_classes = unique(image_pred_[:, -1])

    for cls in img_classes:
      cls_mask = image_pred_ * (image_pred_[:, -1] == cls).float().unsqueeze(1)
      class_mask_ind = torch.nonzero(cls_mask[:,-2]).squeeze()
      image_pred_class = image_pred_[class_mask_ind].view(-1, 7)

      conf_sort_index = torch.sort(image_pred_class[:,4], descending=True)[1]
      image_pred_class = image_pred_class[conf_sort_index]
      idx = image_pred_class.size(0)

      for i in range(idx):
        try:
          ious = bbox_iou(image_pred_class[i].unsqueeze(0), image_pred_class[i+1:])
        except ValueError: break
        except IndexError: break

        iou_mask = (ious < nms_conf).float().unsqueeze(1)
        image_pred_class[i+1:] *= iou_mask

        non_zero_ind = torch.nonzero(image_pred_class[:,4]).squeeze()
        image_pred_class = image_pred_class[non_zero_ind].view(-1, 7)

      batch_ind = image_pred_class.new(image_pred_class.size(0), 1).fill_(ind)
      seq = batch_ind, image_pred_class

      if not write:
        output = torch.cat(seq, 1)
        write = True
      else:
        out = torch.cat(seq, 1)
        output = torch.cat((output, out))
    
  try: return output
  except: return 0

In [ ]:
def unique(tensor):
  tensor_np = tensor.cpu().numpy()
  unique_np = np.unique(tensor_np)
  unique_tensor = torch.from_numpy(unique_np)

  tensor_res = tensor.new(unique_tensor.shape)
  tensor_res.copy_(unique_tensor)
  return tensor_res

In [ ]:
def bbox_iou(box1, box2):
  b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
  b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]

  inter_rect_x1 = torch.max(b1_x1, b2_x1)
  inter_rect_y1 = torch.max(b1_y1, b2_y1)
  inter_rect_x2 = torch.min(b1_x2, b2_x2)
  inter_rect_y2 = torch.min(b1_y2, b2_y2)

  inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * \
                torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)

  b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
  b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)

  iou = inter_area / (b1_area + b2_area - inter_area)

  return iou

run after converting ipynb to py


In [ ]:
def arg_parse():
  parser = argparse.ArgumentParser(description="YOLO v3 Detection Module")
  parser.add_argument("--images", dest="images", 
                     help="Image / Directory containing images to perform detection upon",
                     default = "imgs", type = str)
  parser.add_argument("--det", dest = "det", 
                      help = "Image / Directory to store detections to",
                      default = "det", type = str)
  parser.add_argument("--bs", dest = "bs", help = "Batch size", default = 1)
  parser.add_argument("--confidence", dest = "confidence", 
                      help="Object Confidence to filter predictions", default = 0.5)
  parser.add_argument("--nms_thresh", dest = "nms_thresh", help = "NMS Threshhold", default = 0.4)
  parser.add_argument("--n_cls", dest = "num_classes", help = "Num of Classes", default = 80)
  parser.add_argument("--coco", dest = "classes", help = "coco data file",
                      default = "data/coco.names", type = str)
  parser.add_argument("--cfg", dest = "cfgfile", help = "config file",
                      default = "cfg/yolov3.cfg", type = str)
  parser.add_argument("--weights", dest = "weightsfile", help = "weightsfile",
                      default = "yolov3.weights", type = str)
  parser.add_argument("--reso", dest = "reso",
                      help = "Input resolution of the network. Increase to increase accuracy. Decrease to increase speed",
                      default = "416", type = str)
  return parser.parse_args()


def load_classes(namesfile):
  fp = open(namesfile, "r")
  names = fp.read().split("\n")[:-1]
  fp.close()
  return names


def letterbox_image(img, inp_dim):
  img_w, img_h = img.shape[1], img.shape[0]
  w, h = inp_dim
  new_w = int(img_w * min(w/img_w, h/img_h))
  new_h = int(img_h * min(w/img_w, h/img_h))
  resized_image = cv2.resize(img, (new_w, new_h), interpolation = cv2.INTER_CUBIC)

  canvas = np.full((inp_dim[1], inp_dim[0], 3), 128)

  canvas[(h-new_h)//2:(h-new_h)//2 + new_h, (w-new_w)//2:(w-new_w)//2 + new_w, :] = resized_image

  return canvas


def prep_image(img, inp_dim):
  img = cv2.resize(img, (inp_dim, inp_dim))
  img = img[:,:,::-1].transpose((2,0,1)).copy()
  img = torch.from_numpy(img).float().div(255.0).unsqueeze(0)
  return img


def write(x, results, color):
  c1 = tuple(x[1:3].int())
  c2 = tuple(x[3:5].int())
  img = results[int(x[0])]
  cls = int(x[-1])
  label = f"{classes[cls]}"
  cv2.retangle(img, c1, c2, color, 1)
  t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1, 1)[0]
  c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
  cv2.rectangle(img, c1, c2, color, -1)
  cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [255,255,255], 1)
  return img

In [ ]:
import argparse

args = arg_parse()
images = args.images
batch_size = int(args.bs)
confidence = float(args.confidence)
nms_thesh = float(args.nms_thresh)
num_classes = int(args.num_classes)
start = 0
classes = load_classes(args.classes)
CUDA = torch.cuda.is_available()

print("Loading network.....", end="")
model = Darknet(args.cfgfile)
model.load_wegihts(args.weightsfile)
print("Network successfully loaded!")
print(f"Network predict using {'cuda' if CUDA else 'cpu'}")

model.net_info["height"] = args.reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0
assert inp_dim > 32

if CUDA: model.cuda()
model.eval()

read_dir = time.time()

try:
  imlist = [osp.join(osp.realpath("."), images, img) for img in os.listdir(images)]
except NotADirectoryError:
  imlist = []
  imlist.append(osp.join(osp.realpath("."), images))
except FileNotFoundError:
  print(f"No file or directory with the name {images}")
  exit()

if not os.path.exists(args.det):
  os.makedirs(args.det)

load_batch = time.time()
loaded_ims = [cv2.imread(x) for x in imlist]

im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))

im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1, 2)

if CUDA: im_dim_list = im_dim_list.cuda()

leftover = 0
if (len(im_dim_list) % batch_size):
  leftover = 1

if batch_size != 1:
  num_batches = len(imlist) // batch_size + leftover
  im_batches = [torch.cat((im_batches[i*batch_size : min((i + 1)*batch_size,
                  len(im_batches))])) for i in range(num_batches)]

write = 0
start_det_loop = time.time()
for i, batch in enumerate(im_batches):
  start = time.time()
  if CUDA: batch = batch.cuda()

  prediction = model(Variable(batch, volatile = True), CUDA)

  prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thresh)

  end = time.time()

  if type(prediction) == int:

    for im_num, image in enumerate(imlist[i*batch_size: min((i+1)*batch_size, len(imlist))]):
      im_id = i*batch_size + im_num
      print("{0:20s} prdicted in {1:6.3f} seconds".format(image.split("/")[-1], (end-start)/batch_size))
      print("{0:20s} {1:s}".format("Objects Detected:", ""))
      print("-"*50)

    continue
  
  prediction[:, 0] += i*batch_size

  if not write:
    output = prediction
    write = 1
  else:
    output = torch.cat((output, prediction))

  for im_num, image in enumerate(imlist[i*batch_size: min((i+1)*batch_size, len(imlist))]):
    im_id = i*batch_size + im_num
    objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
    print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end-start)/batch_size))
    print("{0:20s} {1:s}".format("Objects Detected:"," ".join(objs)))
    print("-"*50)

  if CUDA: torch.cuda.synchronize()

try: output
except NameError:
  print("No detections were made")
  exit()

im_dim_list = torch.index_select(im_dim_list, 0, output[:, 0].long())
scaling_factor = torch.min(inp_dim/im_dim_list, 1)[0].view(-1, 1)

output[:,[1, 3]] -= (inp_dim - scaling_factor*im_dim_list[:, 0].view(-1, 1))/2
output[:,[2, 4]] -= (inp_dim - scaling_factor*im_dim_list[:, 1].view(-1, 1))/2

output[:, 1:5] /= scaling_factor

for i in range(output.shape[0]):
  output[i, [1, 3]] = torch.clamp(output[i, [1, 3]], 0.0, im_dim_list[i, 0])
  output[i, [2, 4]] = torch.clamp(output[i, [2, 4]], 0.0, im_dim_list[i, 1])

class_load = time.time()
colors = pkl.load(open("/content/YOLO_v3_tutorial_from_scratch/pallete", "rb"))

draw = time.time()

list(map(lambda x: write(x, loaded_ims, colors), output))

det_names = pd.Series(imlist).apply(lambda x: f"{args.det}/det_{x.split('/')[-1]}")

list(map(cv2.imwrite, det_names, loaded_ims))
end = time.time()

print("SUMMARY")
print("-"*50)
print("{:25s}: {}".format("Task", "Time Taken (in seconds)"))
print()
print("{:25s}: {:2.3f}".format("Reading addresses", load_batch - read_dir))
print("{:25s}: {:2.3f}".format("Loading batch", start_det_loop - load_batch))
print("{:25s}: {:2.3f}".format("Detection (" + str(len(imlist)) + " images", output_recast - start_det_loop))
print("{:25s}: {:2.3f}".format("Output Processing", class_load - output_recast))
print("{:25s}: {:2.3f}".format("Drawing Boxes", end - draw))
print("{:25s}: {:2.3f}".format("Average time_per_img", (end - load_batch)/len(imlist)))
print("-"*50)

torch.cuda.empty_cache()

usage: ipykernel_launcher.py [-h] [--images IMAGES] [--det DET] [--bs BS]
                             [--confidence CONFIDENCE]
                             [--nms_thresh NMS_THRESH] [--n_cls NUM_CLASSES]
                             [--coco CLASSES] [--cfg CFGFILE]
                             [--weights WEIGHTSFILE] [--reso RESO]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-bdd35cc8-5401-4d44-af90-b5fc4bf037bd.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


https://github.com/WongKinYiu/ScaledYOLOv4

In [ ]:
!git clone https://github.com/WongKinYiu/ScaledYOLOv4.git